# core

> Cache your API calls and make your notebooks fast again.

### Introduction

We often call APIs while prototyping and testing our code. A single API call (e.g. an Anthropic chat completion) can take 100's of ms to run. This can really slow down development especially if our notebook contains many API calls 😞.

`cachy` caches API requests. It does this by saving the result of each API call to a local `cachy.jsonl` file. Before calling an API (e.g. OpenAI) it will check if the request already exists in `cachy.jsonl`. If it does it will return the cached result.

**How does it work?**

Under the hood popular SDK's like OpenAI, Anthropic and LiteLLM use `httpx.Client` and `httpx.AsyncClient`. 

`cachy` patches the `send` method of both clients and injects a simple caching mechanism:

- create a cache key from the request
- if the key exists in `cachy.jsonl` return the cached response
- if not, call the API and save the response to `cachy.jsonl`

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import hashlib,httpx,json
from fastcore.utils import *

`cachy.jsonl` contains one API response per line. 

Each line has the following format `{"key": key, "response": response}` 

- `key`: hash of the API request
- `response`: the API response. 

```json
{
    "key": "afc2be0c", 
    "response": "{\"id\":\"msg_xxx\",\"type\":\"message\",\"role\":\"assistant\",\"model\":\"claude-sonnet-4-20250514\",\"content\":[{\"type\":\"text\",\"text\":\"Coordination.\"}],\"stop_reason\":\"end_turn\",\"stop_sequence\":null,\"usage\":{\"input_tokens\":16,\"cache_creation_input_tokens\":0,\"cache_read_input_tokens\":0,\"cache_creation\":{\"ephemeral_5m_input_tokens\":0,\"ephemeral_1h_input_tokens\":0},\"output_tokens\":6,\"service_tier\":\"standard\"}}"
}
```

### Patching `httpx`

Patching a method is very straightforward. 

In our case we want to patch `httpx._client.Client.send` and `httpx._client.AsyncClient.send`. 

These methods are called when running `httpx.get`, `httpx.post`, etc. 

In the example below we use `@patch` from [fastcore](https://fastcore.fast.ai/) to print `calling an API` when `httpx._client.Client.send` is run.

```python
@patch
def send(self:httpx._client.Client, r, **kwargs):
    print('calling an API')
    return self._orig_send(r, **kwargs)
```

### Cache Filtering

Now, let's build up our caching logic piece-by-piece.

The first thing we need to do is ensure that our caching logic only runs on specific urls.

For now, let's only cache API calls made to popular LLM providers like OpenAI, Anthropic, Google and DeepSeek. We can make this fully customizable later.

In [ ]:
#| exports
doms = ("api.openai.com", "api.anthropic.com", "generativelanguage.googleapis.com", "api.deepseek.com")

In [ ]:
#| exports
def _should_cache(url, doms): return any(dom in str(url) for dom in doms)

We could then use `_should_cache` like this.

```python
@patch
def send(self:httpx._client.Client, r, **kwargs):
    if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
    # insert caching logic
    ...
```

### Cache Key

The next thing we need to do is figure out if a response for the request `r` already exists in our cache. 

Recall that each line in `cachy.jsonl` has the following format `{"key": key, "response": response}`.

Our key needs to be unique and deterministic. One way to do this is to concatenate the request URL and content, then generate a hash from the result.

In [ ]:
def _key(r): return hashlib.sha256(str(r.url).encode() + r.content).hexdigest()[:8]

Let's test this out.

In [ ]:
r1 = httpx.Request('POST', 'https://answer.ai', content=b'some content')
r1

<Request('POST', 'https://answer.ai')>

In [ ]:
_key(r1)

'3baa91c4'

If we run it again we should get the same key.

In [ ]:
_key(r1)

'3baa91c4'

Let's modify the url and confirm we get a different key.

In [ ]:
_key(httpx.Request('POST', 'https://fast.ai/', content=b'some content'))

'afee66b2'

Great. Let's update our patch.

```python
@patch
def send(self:httpx._client.Client, r, **kwargs):
    if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
    key = _key(r)
    # if cache hit return the response
    # else run the request, write to response the cache and return it
    ...
```

### Cache Reads/Writes

Now let's add some methods that will read from and write to `cachy.jsonl`.

In [ ]:
#| exports
def _cache(key, cfp):
    with open(cfp, "r") as f:
        line = first(f, lambda l: json.loads(l)["key"] == key)
        return json.loads(line)["response"] if line else None

In [ ]:
#| exports
def _write_cache(key, content, cfp):
    with open(cfp, "a") as f: f.write(json.dumps({"key":key, "response": content})+"\n")

Let's update our `patch`.

In [ ]:
@patch
def send(self:httpx._client.Client, r, **kwargs):
    if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
    key = key(r)
    if res := _cache(key,"cachy.jsonl"): return httpx.Response(status_code=200, content=res, request=r)
    res = self._orig_send(r, **kwargs)
    content = res.read().decode()
    _write_cache(key, content, "cachy.jsonl")
    return httpx.Response(status_code=res.status_code, content=content, request=r)

### Streaming

Let's add support for streaming. 

First let's include an `is_stream` bool in our hash so that a non-streamed request will generate a different key to the same request when streamed. 

In [ ]:
#| exports
def _key(r, is_stream=False):
    "Create a unique, deterministic id from the request `r`."
    return hashlib.sha256(f"{r.url}{is_stream}".encode() + r.content).hexdigest()[:8]

In the `patch` we need to `consume` the entire stream before writing it to the cache.

In [ ]:
@patch
def send(self:httpx._client.Client, r, **kwargs):
    is_stream = kwargs.get("stream")
    if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
    key = _key(r, is_stream=False)
    if res := _cache(key,"cachy.jsonl"): return httpx.Response(status_code=200, content=res, request=r)
    res = self._orig_send(r, **kwargs)
    content = res.read().decode() if not is_stream else b''.join(list(res.iter_bytes())).decode()
    _write_cache(key, content, "cachy.jsonl")
    return httpx.Response(status_code=res.status_code, content=content, request=r)

### `enable_cachy` 

To make `cachy` as user friendly as possible let's make it so that we can apply our patch by running a single method at the top of our notebook.

```python
from cachy import enable_cachy

enable_cachy()
```

For this to work we'll need to wrap our patch.

In [ ]:
def _apply_patch():    
    @patch    
    def send(self:httpx._client.Client, r, **kwargs):
        is_stream = kwargs.get("stream")
        if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
        key = _key(r, is_stream=False)
        if res := _cache(key,"cachy.jsonl"): return httpx.Response(status_code=200, content=res, request=r)
        res = self._orig_send(r, **kwargs)
        content = res.read().decode() if not is_stream else b''.join(list(res.iter_bytes())).decode()
        _write_cache(key, content, "cachy.jsonl")
        return httpx.Response(status_code=res.status_code, content=content, request=r)

In [ ]:
def enable_cachy():  
    _apply_patch()

Great. Now, let's make `cachy` a little more customizable by making it possible to specify:

- the APIs (or domains) to cache
- the location of the cache file.

In [ ]:
def enable_cachy(cache_dir=None, doms=doms):
    cfp = Path(cache_dir or Config.find("settings.ini").config_path or ".") / "cachy.jsonl"
    cfp.touch(exist_ok=True)   
    _apply_patch(cfp, doms)

*Note: If our notebook is running in an nbdev project `Config.find("settings.ini").config_path` automatically finds the base dir.*

In [ ]:
def _apply_patch(cfp, doms):    
    @patch
    def send(self:httpx._client.Client, r, **kwargs):
        is_stream = kwargs.get("stream")
        if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
        key = _key(r, is_stream=False)
        if res := _cache(key,cfp): return httpx.Response(status_code=200, content=res, request=r)
        res = self._orig_send(r, **kwargs)
        content = res.read().decode() if not is_stream else b''.join(list(res.iter_bytes())).decode()
        _write_cache(key,content,cfp)
        return httpx.Response(status_code=res.status_code, content=content, request=r)

### Async

Now let's add support for `async` requests.

In [ ]:
#| exports
def _apply_async_patch(cfp, doms):    
    @patch
    async def send(self:httpx._client.AsyncClient, r, **kwargs):
        is_stream = kwargs.get("stream")
        if not _should_cache(r.url, doms): return await self._orig_send(r, **kwargs)
        key = _key(r, is_stream=False)
        if res := _cache(key, cfp): return httpx.Response(status_code=200, content=res, request=r)
        res = await self._orig_send(r, **kwargs)
        content = res.read().decode() if not is_stream else b''.join([c async for c in res.aiter_bytes()]).decode()
        _write_cache(key, content, cfp)
        return httpx.Response(status_code=res.status_code, content=content, request=r)

Let's rename our original patch.

In [ ]:
#| exports
def _apply_sync_patch(cfp, doms):    
    @patch
    def send(self:httpx._client.Client, r, **kwargs):
        is_stream = kwargs.get("stream")
        if not _should_cache(r.url, doms): return self._orig_send(r, **kwargs)
        key = _key(r, is_stream=False)
        if res := _cache(key,cfp): return httpx.Response(status_code=200, content=res, request=r)
        res = self._orig_send(r, **kwargs)
        content = res.read().decode() if not is_stream else b''.join(list(res.iter_bytes())).decode()
        _write_cache(key,content,cfp)
        return httpx.Response(status_code=res.status_code, content=content, request=r)

Finally, let's update `enable_cachy`.

In [ ]:
#| exports
def enable_cachy(cache_dir=None, doms=doms):
    cfp = Path(cache_dir or Config.find("settings.ini").config_path or ".") / "cachy.jsonl"
    cfp.touch(exist_ok=True)   
    _apply_sync_patch(cfp, doms)
    _apply_async_patch(cfp, doms)

## Tests

Let's test `enable_cachy` on 3 SDKs (OpenAI, Anthropic, LiteLLM) for the scenarios below:

- sync requests with(out) streaming
- async requests with(out) streaming

Add some helper functions.

In [ ]:
class mods: ant="claude-sonnet-4-20250514"; oai="gpt-4o"; gem="gemini/gemini-2.0-flash"

In [ ]:
def mk_msgs(m): return [{"role": "user", "content": f"write 1 word about {m}"}]

In [ ]:
enable_cachy()

### OpenAI

In [ ]:
from openai import OpenAI

In [ ]:
cli = OpenAI()

In [ ]:
r = cli.responses.create(model=mods.oai, input=mk_msgs("openai sync"))
r

Collaboration

<details>

- id: resp_68b98c0b9cac819f95bbf6ad7ee7cabd0ae9bf0ceab25340
- created_at: 1756990475.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4o-2024-08-06
- object: response
- output: [ResponseOutputMessage(id='msg_68b98c0c6fdc819fa02a8a8152719d670ae9bf0ceab25340', content=[ResponseOutputText(annotations=[], text='Collaboration', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: []
- top_p: 1.0
- background: False
- conversation: None
- max_output_tokens: None
- max_tool_calls: None
- previous_response_id: None
- prompt: None
- prompt_cache_key: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- safety_identifier: None
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium')
- top_logprobs: 0
- truncation: disabled
- usage: ResponseUsage(input_tokens=15, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=3, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=18)
- user: None
- store: True

</details>

In [ ]:
r = cli.responses.create(model=mods.oai, input=mk_msgs("openai sync"))
r

Collaboration

<details>

- id: resp_68b98c0b9cac819f95bbf6ad7ee7cabd0ae9bf0ceab25340
- created_at: 1756990475.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4o-2024-08-06
- object: response
- output: [ResponseOutputMessage(id='msg_68b98c0c6fdc819fa02a8a8152719d670ae9bf0ceab25340', content=[ResponseOutputText(annotations=[], text='Collaboration', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: []
- top_p: 1.0
- background: False
- conversation: None
- max_output_tokens: None
- max_tool_calls: None
- previous_response_id: None
- prompt: None
- prompt_cache_key: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- safety_identifier: None
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium')
- top_logprobs: 0
- truncation: disabled
- usage: ResponseUsage(input_tokens=15, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=3, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=18)
- user: None
- store: True

</details>

Let's test streaming.

In [ ]:
r = cli.responses.create(model=mods.oai, input=mk_msgs("openai sync streaming"), stream=True)
for ch in r: print(ch)

ResponseCreatedEvent(response=Response(id='resp_68b98c1027808194a1740c52e02008b00b23cab13df8b32a', created_at=1756990480.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_68b98c1027808194a1740c52e02008b00b23cab13df8b32a', created_at=1756990480.0, error=None, incomplete_details=None, instructions=No

In [ ]:
r = cli.responses.create(model=mods.oai, input=mk_msgs("openai sync streaming"), stream=True)
for ch in r: print(ch)

ResponseCreatedEvent(response=Response(id='resp_68b98c1027808194a1740c52e02008b00b23cab13df8b32a', created_at=1756990480.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_68b98c1027808194a1740c52e02008b00b23cab13df8b32a', created_at=1756990480.0, error=None, incomplete_details=None, instructions=No

Let's test async.

In [ ]:
from openai import AsyncOpenAI

In [ ]:
cli = AsyncOpenAI()

In [ ]:
r = await cli.responses.create(model=mods.oai, input=mk_msgs("openai async"))
r

Innovative

<details>

- id: resp_68b98c1d2fa881a2af14282ad1943b5b01fd7389a58824ea
- created_at: 1756990493.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4o-2024-08-06
- object: response
- output: [ResponseOutputMessage(id='msg_68b98c1debfc81a298eb9da76ec1c55401fd7389a58824ea', content=[ResponseOutputText(annotations=[], text='Innovative', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: []
- top_p: 1.0
- background: False
- conversation: None
- max_output_tokens: None
- max_tool_calls: None
- previous_response_id: None
- prompt: None
- prompt_cache_key: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- safety_identifier: None
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium')
- top_logprobs: 0
- truncation: disabled
- usage: ResponseUsage(input_tokens=15, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=3, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=18)
- user: None
- store: True

</details>

In [ ]:
r = await cli.responses.create(model=mods.oai, input=mk_msgs("openai async"))
r

Innovative

<details>

- id: resp_68b98c1d2fa881a2af14282ad1943b5b01fd7389a58824ea
- created_at: 1756990493.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4o-2024-08-06
- object: response
- output: [ResponseOutputMessage(id='msg_68b98c1debfc81a298eb9da76ec1c55401fd7389a58824ea', content=[ResponseOutputText(annotations=[], text='Innovative', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: []
- top_p: 1.0
- background: False
- conversation: None
- max_output_tokens: None
- max_tool_calls: None
- previous_response_id: None
- prompt: None
- prompt_cache_key: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- safety_identifier: None
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium')
- top_logprobs: 0
- truncation: disabled
- usage: ResponseUsage(input_tokens=15, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=3, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=18)
- user: None
- store: True

</details>

Let's test async streaming.

In [ ]:
r = await cli.responses.create(model=mods.oai, input=mk_msgs("openai async streaming"), stream=True)
async for ch in r: print(ch)

ResponseCreatedEvent(response=Response(id='resp_68b98c217c14819e9206cffe640fb615057b4550d4f1f4e1', created_at=1756990497.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_68b98c217c14819e9206cffe640fb615057b4550d4f1f4e1', created_at=1756990497.0, error=None, incomplete_details=None, instructions=No

In [ ]:
r = await cli.responses.create(model=mods.oai, input=mk_msgs("openai async streaming"), stream=True)
async for ch in r: print(ch)

ResponseCreatedEvent(response=Response(id='resp_68b98c217c14819e9206cffe640fb615057b4550d4f1f4e1', created_at=1756990497.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_68b98c217c14819e9206cffe640fb615057b4550d4f1f4e1', created_at=1756990497.0, error=None, incomplete_details=None, instructions=No

### Anthropic

In [ ]:
from anthropic import Anthropic

In [ ]:
cli = Anthropic()

In [ ]:
r = cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant sync"))
r

**Coordination**

<details>

- id: `msg_0181qZnqWn18VFK8JtqMAgWk`
- content: `[{'citations': None, 'text': '**Coordination**', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 7, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
r = cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant sync"))
r

**Coordination**

<details>

- id: `msg_0181qZnqWn18VFK8JtqMAgWk`
- content: `[{'citations': None, 'text': '**Coordination**', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 7, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Let's test streaming.

In [ ]:
r = cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant sync streaming"), stream=True)
for ch in r: print(ch)

RawMessageStartEvent(message=Message(id='msg_01Xru7ANUjkez8Zwje9uV175', content=[], model='claude-sonnet-4-20250514', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=In: 16; Out: 1; Cache create: 0; Cache read: 0; Total Tokens: 17; Search: 0), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='**', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='Latency**', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockStopEvent(index=0, type='content_block_stop')
RawMessageDeltaEvent(delta=Delta(stop_reason='end_turn', stop_sequence=None), type='message_delta', usage=MessageDeltaUsage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=16, output_tokens=8, server_tool_use=None))
RawMessageStopEvent(type='message_stop')


In [ ]:
r = cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant sync streaming"), stream=True)
for ch in r: print(ch)

RawMessageStartEvent(message=Message(id='msg_01Xru7ANUjkez8Zwje9uV175', content=[], model='claude-sonnet-4-20250514', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=In: 16; Out: 1; Cache create: 0; Cache read: 0; Total Tokens: 17; Search: 0), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='**', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='Latency**', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockStopEvent(index=0, type='content_block_stop')
RawMessageDeltaEvent(delta=Delta(stop_reason='end_turn', stop_sequence=None), type='message_delta', usage=MessageDeltaUsage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=16, output_tokens=8, server_tool_use=None))
RawMessageStopEvent(type='message_stop')


Let's test async.

In [ ]:
from anthropic import AsyncAnthropic

In [ ]:
cli = AsyncAnthropic()

In [ ]:
r = await cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant async"))
r

**Concurrent**

<details>

- id: `msg_01WC1453hRB7KXaMhzTC6FJ8`
- content: `[{'citations': None, 'text': '**Concurrent**', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 7, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
r = await cli.messages.create(model=mods.ant, max_tokens=1024, messages=mk_msgs("ant async"))
r

**Concurrent**

<details>

- id: `msg_01WC1453hRB7KXaMhzTC6FJ8`
- content: `[{'citations': None, 'text': '**Concurrent**', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 7, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Let's test async streaming.

In [ ]:
r = await cli.messages.create(model=mods.ant,max_tokens=1024,messages=mk_msgs("ant async streaming"), stream=True)
async for ch in r.response.aiter_bytes(): print(ch.decode())

event: message_start
data: {"type":"message_start","message":{"id":"msg_01AiTdgVvEzbkE5mbBrSC5Lp","type":"message","role":"assistant","model":"claude-sonnet-4-20250514","content":[],"stop_reason":null,"stop_sequence":null,"usage":{"input_tokens":16,"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"cache_creation":{"ephemeral_5m_input_tokens":0,"ephemeral_1h_input_tokens":0},"output_tokens":1,"service_tier":"standard"}}         }

event: content_block_start
data: {"type":"content_block_start","index":0,"content_block":{"type":"text","text":""}    }

event: content_block_delta
data: {"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Async"}           }

event: content_block_delta
data: {"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Iterable"}     }

event: content_block_stop
data: {"type":"content_block_stop","index":0            }

event: message_delta
data: {"type":"message_delta","delta":{"stop_reason":"end_turn","stop

In [ ]:
r = await cli.messages.create(model=mods.ant,max_tokens=1024,messages=mk_msgs("ant async streaming"), stream=True)
async for ch in r.response.aiter_bytes(): print(ch.decode())

event: message_start
data: {"type":"message_start","message":{"id":"msg_01AiTdgVvEzbkE5mbBrSC5Lp","type":"message","role":"assistant","model":"claude-sonnet-4-20250514","content":[],"stop_reason":null,"stop_sequence":null,"usage":{"input_tokens":16,"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"cache_creation":{"ephemeral_5m_input_tokens":0,"ephemeral_1h_input_tokens":0},"output_tokens":1,"service_tier":"standard"}}         }

event: content_block_start
data: {"type":"content_block_start","index":0,"content_block":{"type":"text","text":""}    }

event: content_block_delta
data: {"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Async"}           }

event: content_block_delta
data: {"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Iterable"}     }

event: content_block_stop
data: {"type":"content_block_stop","index":0            }

event: message_delta
data: {"type":"message_delta","delta":{"stop_reason":"end_turn","stop

### LiteLLM

Let's test the LiteLLM SDK by running sync/async calls with(out) streaming for OpenAI, Anthropic, & Gemini.

We'll also double check tool calls and citations.

#### Sync Tests

In [ ]:
from litellm import completion

Let's define a helper method to display a streamed response.

In [ ]:
def _stream(r): 
    for ch in r: print(ch.choices[0].delta.content or "")

##### Anthropic

Let's test `claude-sonnet-x`.

In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("lite: ant sync..."))
r

ModelResponse(id='chatcmpl-2d7bc3aa-c8f4-4d24-90dd-f496ea5345ca', created=1756990536, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='seamless', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=5, prompt_tokens=18, total_tokens=23, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("lite: ant sync..."))
r

ModelResponse(id='chatcmpl-6c9f6f82-613c-43d5-8f69-e0992e47a4b9', created=1756990539, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='seamless', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=5, prompt_tokens=18, total_tokens=23, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

Now, with streaming enabled.

In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("lite: ant sync stream..."), stream=True)
_stream(r)

**
Stream**

(It flows with minimal overhea
d, like a lightweight synchronous data stream connecting
 distributed processes)



In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("lite: ant sync stream..."), stream=True)
_stream(r)

**
Stream**

(It flows with minimal overhea
d, like a lightweight synchronous data stream connecting
 distributed processes)



##### OpenAI

Let's test `gpt-4o`.

In [ ]:
r = completion(model=mods.oai, messages=mk_msgs("lite: oai sync..."))
r

ModelResponse(id='chatcmpl-CC3nFvP1pBj3wuyFBKDq4E4awiP9G', created=1756990553, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_cbf1785567', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Efficiency.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=2, prompt_tokens=18, total_tokens=20, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

In [ ]:
r = completion(model=mods.oai, messages=mk_msgs("lite: oai sync..."))
r

ModelResponse(id='chatcmpl-CC3nFvP1pBj3wuyFBKDq4E4awiP9G', created=1756990553, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_cbf1785567', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Efficiency.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=2, prompt_tokens=18, total_tokens=20, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

Now, with streaming enabled.

In [ ]:
r = completion(model=mods.oai, messages=mk_msgs("lite: oai sync stream..."), stream=True)
_stream(r)

Integration




In [ ]:
r = completion(model=mods.oai, messages=mk_msgs("lite: oai sync stream..."), stream=True)
_stream(r)

Integration




##### Gemini

Let's test `2.0-flash`.

In [ ]:
r = completion(model=mods.gem, messages=mk_msgs("lite: gem sync..."))
r

ModelResponse(id='Yoy5aJ-NGN78nsEP9J-5gQ0', created=1756990562, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Speed\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=2, prompt_tokens=10, total_tokens=12, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=10, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_url_context_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])

In [ ]:
r = completion(model=mods.gem, messages=mk_msgs("lite: gem sync..."))
r

ModelResponse(id='Yoy5aJ-NGN78nsEP9J-5gQ0', created=1756990563, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Speed\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=2, prompt_tokens=10, total_tokens=12, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=10, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_url_context_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])

Now, with streaming enabled.

In [ ]:
r = completion(model=mods.gem, messages=mk_msgs("lite: gem sync stream..."), stream=True)
_stream(r)

Real
time




In [ ]:
r = completion(model=mods.gem, messages=mk_msgs("lite: gem sync stream..."), stream=True)
_stream(r)

Real
time




#### Async Tests

In [ ]:
from litellm import acompletion

In [ ]:
async def _astream(r):
    async for chunk in r: print(chunk.choices[0].delta.content or "")

##### Anthropic

Let's test `claude-sonnet-x`.

In [ ]:
r = await acompletion(model=mods.ant, messages=mk_msgs("lite: ant async..."))
r

ModelResponse(id='chatcmpl-646ca947-93e1-4e6d-bc5a-10e7508b0f9a', created=1756990575, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='**concurrent**', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=6, prompt_tokens=18, total_tokens=24, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

In [ ]:
r = await acompletion(model=mods.ant, messages=mk_msgs("lite: ant async..."))
r

ModelResponse(id='chatcmpl-d2bae8f2-5291-418f-bb7f-42e851a169e1', created=1756990577, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='**concurrent**', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=6, prompt_tokens=18, total_tokens=24, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

Now, with streaming enabled.

In [ ]:
r = await acompletion(model=mods.ant, messages=mk_msgs("lite: ant async stream..."), stream=True)
await(_astream(r))

**
Generator**



In [ ]:
r = await acompletion(model=mods.ant, messages=mk_msgs("lite: ant async stream..."), stream=True)
await(_astream(r))

**
Generator**



##### OpenAI

Let's test `gpt-4o`.

In [ ]:
r = await acompletion(model=mods.oai, messages=mk_msgs("lite: oai async..."))
r

ModelResponse(id='chatcmpl-CC3nlRg0j6UCRX5XpCvk3qtilPPPC', created=1756990585, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_80956533cb', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Efficient.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=3, prompt_tokens=18, total_tokens=21, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

In [ ]:
r = await acompletion(model=mods.oai, messages=mk_msgs("lite: oai async..."))
r

ModelResponse(id='chatcmpl-CC3nlRg0j6UCRX5XpCvk3qtilPPPC', created=1756990585, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_80956533cb', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Efficient.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=3, prompt_tokens=18, total_tokens=21, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

Now, with streaming enabled.

In [ ]:
r = await acompletion(model=mods.oai, messages=mk_msgs("lite: oai async stream..."), stream=True)
await(_astream(r))

Eff
icient




In [ ]:
r = await acompletion(model=mods.oai, messages=mk_msgs("lite: oai async stream..."), stream=True)
await(_astream(r))

Eff
icient




##### Gemini

Let's test `2.0-flash`.

In [ ]:
r = await acompletion(model=mods.gem, messages=mk_msgs("lite: gem async..."))
r

ModelResponse(id='goy5aNKBKZfjnsEPsPWEmA0', created=1756990594, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Fiber\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=2, prompt_tokens=10, total_tokens=12, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=10, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_url_context_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])

In [ ]:
r = await acompletion(model=mods.gem, messages=mk_msgs("lite: gem async..."))
r

ModelResponse(id='goy5aNKBKZfjnsEPsPWEmA0', created=1756990596, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Fiber\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=2, prompt_tokens=10, total_tokens=12, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=10, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_url_context_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])

Now, with streaming enabled.

In [ ]:
r = await acompletion(model=mods.gem, messages=mk_msgs("lite: gem async stream..."), stream=True)
await(_astream(r))

Efficient



In [ ]:
r = await acompletion(model=mods.gem, messages=mk_msgs("lite: gem async stream..."), stream=True)
await(_astream(r))

Efficient



#### Tool Calls

As a sanity check let's confirm that tool calls work.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type":"string", "description":"The city e.g. Reims"},
                    "unit": {"type":"string", "enum":["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            }
        }
    }
]

In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("Is it raining in Reims?"), tools=tools)
r

ModelResponse(id='chatcmpl-3ac7d40f-81d0-4c72-8f34-34564d1ddbc4', created=1756990637, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"location": "Reims"}', name='get_current_weather'), id='toolu_01GVccciafRHRKcb6JoQbu38', type='function')], function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=57, prompt_tokens=427, total_tokens=484, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

In [ ]:
r = completion(model=mods.ant, messages=mk_msgs("Is it raining in Reims?"), tools=tools)
r

ModelResponse(id='chatcmpl-e7179c73-f158-4fc1-b5cb-002ec9059b5b', created=1756990638, model='claude-sonnet-4-20250514', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"location": "Reims"}', name='get_current_weather'), id='toolu_01GVccciafRHRKcb6JoQbu38', type='function')], function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=57, prompt_tokens=427, total_tokens=484, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

## Export -

In [ ]:
#|hide
#|eval: false
import nbdev; nbdev.nbdev_export()